## Análisis de datos de la estación de Grazalema

Activación del proyecto de Julia

In [1]:
using Pkg
Pkg.activate(@__DIR__);
Pkg.instantiate()

using StatsPlots
using CSV, DataFrames

import Dates: DateTime

  Activating project at `c:\Users\ayats\Documents\ici-parsing`


Se leen los datos de la salida del sensor en CSV. Para este estudio sólo se va a elegir un rango de los datos para poder visualizar los resultados de forma más fácil. Como los datos vienen recogidos hora a hora, simplemente se escogen los 10 primeros días.

In [2]:
data = CSV.File(
    open("data/Grazalema_AEMET_2021-2022.csv");
    normalizenames=true,
    select=[:ts, :value],
    types=Dict(:ts => DateTime),
    dateformat="yyyy-mm-dd HH:MM:SS.s"
) |> DataFrame

data = data[1:24*10, :]

Row,value,ts
,Float64,DateTime
1,5.4,2021-01-01T01:00:00
2,4.2,2021-01-01T02:00:00
3,4.1,2021-01-01T03:00:00
4,4.1,2021-01-01T04:00:00
5,3.8,2021-01-01T05:00:00
6,3.3,2021-01-01T06:00:00
7,2.7,2021-01-01T07:00:00
8,2.9,2021-01-01T08:00:00
9,3.6,2021-01-01T09:00:00


Se dibujan los datos recogidos directamente

In [3]:
p0 = @df data plot(:ts, :value, 
    label="Measured",
    marker=:circle,
    markersize=2
)

macro plot_common()
    quote
        ylims!(-5,12)
        xlabel!("Date") 
        ylabel!("Temperature (°C)")
    end
end

@plot_common()

Por último, se usa una librería de _singular spectrum analysis_ para estimar una separación de la evolución de la temperatura en una componente de tendencia, y componentes de estacionalidad.

Se dibujan todas las señales para poder comprobar como el método es capaz de separar las señales.

In [4]:
import SingularSpectrumAnalysis as SSA

data_trend, data_season = SSA.analyze(
    data[:, :value],
    20,
    robust=true
) 

data_rem = data[:, :value] .- data_trend .- vec(sum(data_season, dims=2))

layout = @layout [a; b; c; d]
p1 = plot(data[:, :ts], data_trend, label="Trend",)
p2 = plot(data[:, :ts], data_season, label="Seasonality", linestyle=:dash)
p3 = plot(data[:, :ts], data_rem, label="Remainder", linestyle=:dot)
plot(p0, p1, p2, p3, layout=layout, link=:both)

plot!(size=(800,800))
@plot_common()

Como trabajo adicional, se podrían predecir las temperaturas continuado los días siguientes con distintos métodos.

El creador de la librería SSA provee de funciones para hacer esta estimación, teniendo los términos de tendencia y estacionalidad, aunque también propone usar librerías de modelos autorregresivos para realizar la estimación.